In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### https://www.druginfo.co.kr/search2/products.aspx?q=Acetylcysteine

In [13]:
k = pd.DataFrame()

response = requests.get('https://www.druginfo.co.kr/search2/products.aspx?q=Acetylcysteine&count=20&sortBy=proKorName&sortOrder=asc')
html = response.text
soup = BeautifulSoup(html, 'html.parser')

tmp = soup.select('#main > div.res.table-res > table')
tmp = str(tmp)
table_list = pd.read_html(tmp)

k = pd.concat([k, table_list[0]], axis = 1)
print(k)

    0                         1    2          3             4   \
0  NaN                     제품명 ▲   임부       보험코드           판매사   
1  NaN            누코미트액(아세틸시스테인)  NaN  693900211     (주)셀트리온제약   
2  NaN      누코미트캡슐200mg(아세틸시스테인)  NaN  693900030     (주)셀트리온제약   
3  NaN            다세틸캡슐(아세틸시스테인)  NaN  658107130       (유)한풍제약   
4  NaN          대웅아세틸시스테인캅셀200mg  NaN  000000000         (주)대웅   
5  NaN     독립목클린캡슐200mg(아세틸시스테인)  NaN  057700010    독립바이오제약(주)   
6  NaN           로스테인캡슐(아세틸시스테인)  NaN  052400770   (주)인트로바이오파마   
7  NaN               리보푸렉스주250ml  NaN  645200751    에이프로젠제약(주)   
8  NaN            목테인캡슐(아세틸시스테인)  NaN  669502390     (주)씨티씨바이오   
9  NaN        목틴캡슐200mg(아세틸시스테인)  NaN  643501030       한미약품(주)   
10 NaN      무코테인캡슐200mg(아세틸시스테인)  NaN  661904030       영풍제약(주)   
11 NaN       무테린캡슐200mg(아세틸시스테인)  NaN  644301260  한국유나이티드제약(주)   
12 NaN      뮤라스텐캡슐200mg(아세틸시스테인)  NaN  625800540      메딕스제약(주)   
13 NaN       뮤라텐캡슐100mg(아세틸시스테인)  NaN  057000190       (주)더유제약   
14 NaN    

In [38]:
t = pd.DataFrame()

for idx in range(20, 130, 20): 
    print(idx)
    response = requests.get('https://www.druginfo.co.kr/search2/products.aspx?q=Acetylcysteine&offset={}&count=20&sortBy=proKorName&sortOrder=asc'.format(idx))
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    tmp = soup.select('#main > div.res.table-res > table')
    tmp = str(tmp)
    table_list = pd.read_html(tmp)

    t = pd.concat([t, table_list[0]], axis = 0)
    # print(t)

20
40
60
80
100
120


In [34]:
t.drop([0], axis = 0, inplace = True)

In [32]:
k.columns = k.iloc[0]
k.drop([0], axis = 0, inplace = True)
k.reset_index(drop = True, inplace = True)

In [39]:
t.columns = t.iloc[0]
t.drop([0], axis = 0, inplace = True)
t.reset_index(drop = True, inplace = True)

In [40]:
k

,NaN,제품명 ▲,임부,보험코드,판매사,성분/함량,구분,보험,약가,조회수,대체,수정
0,NaN,누코미트액(아세틸시스테인),NaN,693900211,(주)셀트리온제약,Acetylcysteine 20g,일반,급여,"1,542원",유료,조회,요청
1,NaN,누코미트캡슐200mg(아세틸시스테인),NaN,693900030,(주)셀트리온제약,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
2,NaN,다세틸캡슐(아세틸시스테인),NaN,658107130,(유)한풍제약,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
3,NaN,대웅아세틸시스테인캅셀200mg,NaN,000000000,(주)대웅,N-Acetyl-L-Cysteine 200mg,일반,비급여,0원,유료,조회,요청
4,NaN,독립목클린캡슐200mg(아세틸시스테인),NaN,057700010,독립바이오제약(주),Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
5,NaN,로스테인캡슐(아세틸시스테인),NaN,052400770,(주)인트로바이오파마,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
6,NaN,리보푸렉스주250ml,NaN,645200751,에이프로젠제약(주),L-valine 630mg 외 14개,전문,비급여,0원,유료,조회,요청
7,NaN,목테인캡슐(아세틸시스테인),NaN,669502390,(주)씨티씨바이오,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
8,NaN,목틴캡슐200mg(아세틸시스테인),NaN,643501030,한미약품(주),Acetylcysteine 200mg,일반,급여,64원,유료,조회,요청
9,NaN,무코테인캡슐200mg(아세틸시스테인),NaN,661904030,영풍제약(주),Acetylcysteine 200mg,일반,비급여,0원,유료,조회,요청


In [41]:
t

,NaN,제품명 ▲,임부,보험코드,판매사,성분/함량,구분,보험,약가,조회수,대체,수정
0,NaN,뮤코나액(아세틸시스테인) 5mL,NaN,654005631,아주약품(주),Acetylcysteine 20g,일반,급여,"1,650원",유료,조회,요청
1,NaN,뮤코다인캡슐200mg(아세틸시스테인),NaN,698501880,(주)테라젠이텍스,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
2,NaN,뮤코레스캡슐200mg(아세틸시스테인),NaN,653102980,(주)비씨월드제약,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
3,NaN,뮤코린캡슐200mg(아세틸시스테인),NaN,623005460,한국신텍스제약(주),Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
4,NaN,뮤코세린캡슐(아세틸시스테인),NaN,649100280,한국휴텍스제약(주),Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
...,...,...,...,...,...,...,...,...,...,...,...,...
103,NaN,해그린캡슐(아세틸시스테인),NaN,644003410,삼익제약(주),Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
104,NaN,헬파워솔주100ml(bag),NaN,644703830,에스케이케미칼(주),L-valine 630mg 외 14개,전문,비급여,0원,유료,조회,요청
105,NaN,후루케어캡슐200mg(아세틸시스테인)_구.후루무실캡슐200mg,NaN,642902600,일동제약(주),Acetylcysteine 200mg,일반,급여,63원,유료,조회,요청
106,NaN,휴니즈아세틸시스테인캡슐200mg,NaN,670400730,(주)휴온스메디케어,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청


In [42]:
final = pd.concat([k, t], axis = 0)

In [48]:
final.reset_index(drop = True, inplace = True)

In [49]:
final

,NaN,제품명 ▲,임부,보험코드,판매사,성분/함량,구분,보험,약가,조회수,대체,수정
0,NaN,누코미트액(아세틸시스테인),NaN,693900211,(주)셀트리온제약,Acetylcysteine 20g,일반,급여,"1,542원",유료,조회,요청
1,NaN,누코미트캡슐200mg(아세틸시스테인),NaN,693900030,(주)셀트리온제약,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
2,NaN,다세틸캡슐(아세틸시스테인),NaN,658107130,(유)한풍제약,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
3,NaN,대웅아세틸시스테인캅셀200mg,NaN,000000000,(주)대웅,N-Acetyl-L-Cysteine 200mg,일반,비급여,0원,유료,조회,요청
4,NaN,독립목클린캡슐200mg(아세틸시스테인),NaN,057700010,독립바이오제약(주),Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
...,...,...,...,...,...,...,...,...,...,...,...,...
123,NaN,해그린캡슐(아세틸시스테인),NaN,644003410,삼익제약(주),Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청
124,NaN,헬파워솔주100ml(bag),NaN,644703830,에스케이케미칼(주),L-valine 630mg 외 14개,전문,비급여,0원,유료,조회,요청
125,NaN,후루케어캡슐200mg(아세틸시스테인)_구.후루무실캡슐200mg,NaN,642902600,일동제약(주),Acetylcysteine 200mg,일반,급여,63원,유료,조회,요청
126,NaN,휴니즈아세틸시스테인캡슐200mg,NaN,670400730,(주)휴온스메디케어,Acetylcysteine 200mg,일반,급여,81원,유료,조회,요청


In [47]:
final.to_csv('drug_crawling.csv', encoding = 'euc-kr', index = False)